In [18]:
import os
import argparse
from dataloader import load_rcv1_data
from algorithms import *
from utils import save, load, plot
import numpy as np

In [4]:

'''
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', type=float, default=0.01)
    parser.add_argument('--seed', type=int, default=42)
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    here = os.path.dirname(os.path.abspath(__file__))
'''
# Load data
X, y = load_rcv1_data(n_samples=10000, label_idx=0, path='data')
print(f"Data loaded: X shape {X.shape}, y shape {y.shape}")

# Obtain g_opt and f_opt
bound = 50
_, __, g_opt = hinge_loss_minimize(X, y, bound=bound)
print(f'Type of _: {type(_)}')
print(f"Optimal hinge loss: {g_opt:.4f}")
w_opt, b_opt, xi_opt, u_opt, f_opt = L1_norm_second_minimize(X, y, g_opt, bound=bound)
print(f"Optimal upper L1 norm: {f_opt:.4f}")

# Prepare common parameters for algorithms
X = X.toarray()  # Convert to dense array if needed
n, d = X.shape
f_hat = np.linalg.norm(_.copy(), ord=1) / d # upper bound for FC-BiO
R = bound * np.sqrt(d + 1)
Lf = np.sqrt(d) / d
Lg = np.sqrt(np.sum(np.square(np.mean(X, axis=0))) + 1)
L  = max(Lf, Lg)
initial = np.random.uniform(-bound, bound, size=d + 1)  # Initial point for optimization

# Initialize and train BiCS algorithm
bics = BiCS(X, y, L, R, bound, initial, num_iter=1000)
bics.solve(bics.initial, start_iter=1, end_iter=1000)

# Initialize and train FCBiO algorithm
fcbio = FCBiO(X, y, L, bound, initial, T=1000, l=0, u=f_hat, hat_opt_g_x=g_opt, eps=1e-2)
_, fcbio_f, fcbio_g = fcbio.solve()

    

Loaded data from /Users/yj/Desktop/Bilevel-Optimization/dataloader/data/rcv1_10000samples_idx0.npz
Data loaded: X shape (10000, 47236), y shape (10000,)
Set parameter Username
Set parameter LicenseID to value 2685030
Academic license - for non-commercial use only - expires 2026-07-06
Hinge_loss_minimization
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 10000 rows, 57237 columns and 767816 nonzeros
Model fingerprint: 0xf788dc65
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [1e-04, 1e-04]
  Bounds range     [5e+01, 5e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 218 rows and 42989 columns
Presolve time: 0.21s
Presolved: 9782 rows, 14248 columns, 643805 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log on

100%|███████████████████████████████████████| 1000/1000 [03:27<00:00,  4.81it/s]


FC-BiO starts.
─────────────────────────────
[1/14] t = 24.973562


ψ̂(t) = 5.622480e+01 → ↑ l ← t (Runtime: 3.15s)
─────────────────────────────
[2/14] t = 37.460344


ψ̂(t) = 1.487275e+01 → ↑ l ← t (Runtime: 5.84s)
─────────────────────────────
[3/14] t = 43.703734


...

ψ̂(t) = 7.670351e-02 → ↑ l ← t (Runtime: 9.65s)
─────────────────────────────
[11/14] t = 49.922737


ψ̂(t) = 5.223339e-02 → ↑ l ← t (Runtime: 9.58s)
─────────────────────────────
[12/14] t = 49.934931


ψ̂(t) = 3.157584e-02 → ↑ l ← t (Runtime: 9.78s)
─────────────────────────────
[13/14] t = 49.941028


ψ̂(t) = 1.622612e-02 → ↑ l ← t (Runtime: 9.67s)
─────────────────────────────
[14/14] t = 49.944076


ψ̂(t) = 8.283066e-03 → ↑ l ← t (Runtime: 9.60s)
FC-BiO terminated (Total runtime: 125.92s)


In [22]:
from utils import *

In [23]:
# Save results
here = os.getcwd()
save_path = os.path.join(here, f"results/results_1000samples/")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
results = [bics, fcbio]
algorithms = {'0': 'BiCS', '1': 'FCBiO'}
save(save_path, *results, **algorithms)
print(f"Results saved to {save_path}")

# Plot results
plot_path_upper = os.path.join(save_path, f"plot_upper.png")
plot_upper = [np.array(bics.f_plot) - f_opt, fcbio_f - f_opt]
plot(plot_path_upper, 'Upper Gap', *plot_upper, **algorithms)
print(f"Plots saved to {plot_path_upper}")

plot_path_lower = os.path.join(save_path, f"plot_lower.png")
plot_lower = [np.array(bics.g_plot) - g_opt, fcbio_g - g_opt]
plot(plot_path_lower, 'Lower Gap', *plot_lower, **algorithms)
print(f"Plots saved to {plot_path_lower}")

KeyError: 0

In [12]:
def test(**kwargs):
    print(kwargs)

test(**algorithms)

TypeError: keywords must be strings